In [41]:
import numpy as np

class Neuron_net:
    def __init__(self, input_size):
        self.weights = np.random.rand(input_size)
        self.bias = np.random.rand(1)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def feed_forward(self, inputs):
        total = np.dot(inputs, self.weights) + self.bias
        return self.sigmoid(total)

if __name__ == "__main__":
    neuron = Neuron_net(input_size=3)
    inputs = np.array([0.1, 1.4, 4])
    output = neuron.feed_forward(inputs)
    print("Output:", output)

Output: [0.69693744]


In [2]:
import numpy as np         
import pandas as pd

def precision_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    
    if tp + fp == 0:
        return 0.0
    else:
        return tp / (tp + fp)

def load_data():
    train_data=pd.read_csv('mnist_train.csv')
    test_data=pd.read_csv('mnist_test.csv')

    X_train=train_data.iloc[:,1:].values.astype(np.float32)/255.0
    X_test=test_data.iloc[:,1:].values.astype(np.float32)/255.0

    y_train=train_data.iloc[:,0].values
    y_test=test_data.iloc[:,0].values

    y_train_onehot=np.zeros((y_train.size,y_train.max()+1))
    y_train_onehot[np.arange(y_train.size), y_train] = 1

    y_test_onehot=np.zeros((y_test.size,y_test.max()+1))
    y_test_onehot[np.arange(y_test.size), y_test] = 1

    return X_train,y_train_onehot,X_test,y_test_onehot


 
class Neural_network:
    def __init__(self,input_size,hidden_size,output_size,learning_rate,regularization):
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.learning_rate=learning_rate
        self.regularization=regularization

        self.weight_input_hidden=np.random.randn(self.input_size,self.hidden_size)*np.sqrt(2.0 / self.input_size)
        self.bias_hidden=np.zeros((1,self.hidden_size))
        self.weights_hidden_output=np.random.randn(self.hidden_size,self.output_size)*np.sqrt(2.0 / self.hidden_size)
        self.bias_output=np.zeros((1,self.output_size))

    def relu(self,x):
        return np.maximum(0,x)

    def relu_derivative(self,x):
        return np.where(x > 0, 1, 0)

    def softmax(self,x):
        exp_values=np.exp(x-np.max(x,axis=1,keepdims=True))
        return exp_values/np.sum(exp_values,axis=1,keepdims=True)

    def forward(self,X):
        self.hidden_input= np.dot(X,self.weight_input_hidden)+self.bias_hidden
        self.hidden_output=self.relu(self.hidden_input)
        self.final_input=np.dot(self.hidden_output,self.weights_hidden_output)+self.bias_output
        self.final_output=self.softmax(self.final_input)
        return self.final_output

    def backward(self,X,y,output):
        self.error=y-output
        self.output_gradient=self.error

        self.hidden_error=self.output_gradient.dot(self.weights_hidden_output.T)
        self.hidden_gradient=self.hidden_error*self.relu_derivative(self.hidden_output)

        self.weights_hidden_output+=(np.dot(self.hidden_output.T,self.output_gradient)-self.regularization*self.weights_hidden_output)*self.learning_rate
        self.bias_output+=np.sum(self.output_gradient,axis=0,keepdims=True)*learning_rate
        self.weight_input_hidden+=(np.dot(X.T,self.hidden_gradient)-self.regularization*self.weight_input_hidden)*learning_rate
        self.bias_hidden+=np.sum(self.hidden_gradient,axis=0,keepdims=True)*learning_rate


    def train(self,X,y,epochs,batch_size=64):
        sample_size=X.shape[0]
        for epoch in range (epochs):
            shuffle_index=np.random.permutation(sample_size)
            X_shuffled=X[shuffle_index]
            y_shuffled=y[shuffle_index]

            for i in range(0,sample_size,batch_size):
                X_batch=X_shuffled[i:i+batch_size]
                y_batch=y_shuffled[i:i+batch_size]

            output=self.forward(X_batch)
            self.backward(X_batch,y_batch,output)

            if (epoch+1)%10==0:
                train_output=self.forward(X)
                train_loss=-np.mean(np.sum(y*np.log(train_output+1e-8),axis=-1))

                val_output=self.forward(X_test)
                val_loss=-np.mean(np.sum(y_test*np.log(val_output+1e-8),axis=-1))

                print(f'Epoch {epoch+1}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    def predict(self,X):
          output=self.forward(X)
          return np.argmax(output,axis=1)



input_size=784
hidden_size=256
output_size=10
epochs=50
regularization=0.001
learning_rate=0.01

X_train,y_train,X_test,y_test=load_data()
n=Neural_network(input_size,hidden_size,output_size,learning_rate,regularization)

n.train(X_train,y_train,epochs)
predictions=n.predict(X_test)
accuracy=np.mean(predictions==np.argmax(y_test,axis=1))*100
print(f' Accuracy: {accuracy:.2f}%')

precision = np.zeros(output_size)
for cls in range(output_size):
    y_true_cls = (y_test[:, cls] == 1)
    y_pred_cls = (predictions == cls)
    precision[cls] = precision_score(y_true_cls, y_pred_cls)

for i, prec in enumerate(precision):
    print(f'Precision for class {i}: {prec:.4f}')



                
                
                

            
    

Epoch 10, Training Loss: 1.4520, Validation Loss: 1.4370
Epoch 20, Training Loss: 0.7783, Validation Loss: 0.7578
Epoch 30, Training Loss: 0.6441, Validation Loss: 0.6193
Epoch 40, Training Loss: 0.6087, Validation Loss: 0.5883
Epoch 50, Training Loss: 0.5621, Validation Loss: 0.5394
 Accuracy: 82.81%
Precision for class 0: 0.8577
Precision for class 1: 0.9214
Precision for class 2: 0.9591
Precision for class 3: 0.9489
Precision for class 4: 0.9315
Precision for class 5: 0.9147
Precision for class 6: 0.8049
Precision for class 7: 0.9674
Precision for class 8: 0.5997
Precision for class 9: 0.6681


In [11]:
import numpy as np
import pandas as pd


def precision_score(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    
    if tp + fp == 0:
        return 0.0
    else:
        return tp / (tp + fp)



def load_data():
    data = pd.read_csv('data.csv')
    data.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)  # Remove unnecessary columns
    data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]  # Convert categorical labels to numeric

    X = data.drop(['diagnosis'], axis=1).values
    y = data.diagnosis.values

 
    X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))
    
    return X, y


X, y = load_data()

def load_unlabeled_data(data_file):
    data = pd.read_csv(data_file)
    data.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)
    X = data.drop(['diagnosis'], axis=1).values
    X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))
    
    return X


def train_test_split(X, y, test_size=0.2, random_state=None):
    np.random.seed(random_state)
    num_test = int(test_size * len(X))
    indices = np.random.permutation(len(X))
    train_indices, test_indices = indices[num_test:], indices[:num_test]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size) * np.sqrt(2.0 / self.input_size)
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size) * np.sqrt(2.0 / self.hidden_size)
        self.bias_output = np.zeros((1, self.output_size))

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
     
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.relu(self.hidden_input)
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = self.sigmoid(self.final_input)
        return self.final_output

    def backward(self, X, y, output):
        error = y.reshape(-1, 1) - output
        output_gradient = error * output * (1 - output)
        
        hidden_error = np.dot(output_gradient, self.weights_hidden_output.T)
        hidden_gradient = hidden_error * self.relu_derivative(self.hidden_output)
        
        
        self.weights_hidden_output += np.dot(self.hidden_output.T, output_gradient) * self.learning_rate
        self.bias_output += np.sum(output_gradient, axis=0, keepdims=True) * self.learning_rate
        self.weights_input_hidden += np.dot(X.T, hidden_gradient) * self.learning_rate
        self.bias_hidden += np.sum(hidden_gradient, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

            
            if (epoch + 1) % 10 == 0:
                loss = self.calculate_loss(X, y)
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')

    def calculate_loss(self, X, y):
        
        output = self.forward(X)
        loss = -np.mean(y * np.log(output) + (1 - y) * np.log(1 - output))
        
        return loss
        

    def predict(self, X):
        
        output = self.forward(X)
        predictions = (output > 0.5).astype(int)
        return predictions.ravel()


input_size = X_train.shape[1]
hidden_size = 256
output_size = 1  
learning_rate = 0.001
epochs = 100


nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)


nn.train(X_train, y_train, epochs)


predictions = nn.predict(X_test)
accuracy = np.mean(predictions == y_test) * 100
print(f'Test Accuracy: {accuracy:.2f}%')

prec = precision_score(y_test, predictions)
print(f'Precision for class 1: {prec:.4f}')

unlabeled_data_file = 'data.csv'
X_unlabeled = load_unlabeled_data(unlabeled_data_file)


predictions = nn.predict(X_unlabeled)
accuracy = np.mean(predictions == y_test) * 100
print(f'Test Accuracy: {accuracy:.2f}%')

prec = precision_score(y_test, predictions)
print(f'Precision for class 1: {prec:.4f}')





Epoch 10/100, Loss: 0.7016
Epoch 20/100, Loss: 0.7332
Epoch 30/100, Loss: 0.7718
Epoch 40/100, Loss: 0.8112
Epoch 50/100, Loss: 0.8494
Epoch 60/100, Loss: 0.8857
Epoch 70/100, Loss: 0.9200
Epoch 80/100, Loss: 0.9524
Epoch 90/100, Loss: 0.9829
Epoch 100/100, Loss: 1.0118
Test Accuracy: 92.92%
Precision for class 1: 0.9474


ValueError: operands could not be broadcast together with shapes (569,) (113,) 